# Exploring eBay Car Sales Data

In this project we will work on a dataset of used cars from **eBay Kleinanzeigen**, a classifieds section of the German eBay website. The dataset was originally `scraped` and uploaded to Kaggle by user **[orgesleka](https://www.kaggle.com/orgesleka)**. 

The `goal` of this project is `to clean the data and analyze the included used car listings.`

## Table of Contents

> #### 1. Data Sources
> #### 2. Open the dataset
> #### 3. Cleaning Column Names
> #### 4. Initial Exploration and Cleaning
> #### 5. Exploring the Odometer and Price Columns
> #### 6. Exploring the date columns
> #### 7. Dealing with Incorrect Registration Year Data
> #### 8. Exploring Price by Brand
> #### 9. Storing Aggregate Data in a DataFrame
> #### 10. Conclusion

---------------------

### Data Sources
You can find the data set [here](https://data.world/data-society/used-cars-data), but note that we have sampled 50,000 data points from the full dataset which we will be working with. Also the dataset we are working with is dirty to a bit more closely resemble what you would expect from a scraped dataset.

--------------

### Read the dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
autos = pd.read_csv("datasets/autos.csv", encoding="Latin-1")

In [3]:
# display information about the autos dataframe
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [4]:
# display first few rows of the dataframe
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Looking at the information displayed above, we observe that our dataset is made up of `50000 entries(rows)` and `20 columns`, most of which are strings.

    We also observe that some columns have missing or null values. These columns inlcude; `vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage`. However, these null values are less than 20%.

The 20 columns in our dataframe are;

|Column|Description|
|:---|:---|
|dateCrawled  |  When this ad was first crawled. All field-values are taken from this date.|
|name  |  Name of the car.|
|seller  |  Whether the seller is private or a dealer.|
|offerType  |  The type of listing|
|price  |  The price on the ad to sell the car.|
|abtest  |  Whether the listing is included in an A/B test.|
|vehicleType  |  The vehicle Type.|
|yearOfRegistration  |  The year in which the car was first registered.|
|gearbox  |  The transmission type.|
|powerPS  |  The power of the car in PS.|
|model  |  The car model name.|
|odometer  |  How many kilometers the car has driven.|
|monthOfRegistration  |  The month in which the car was first registered.|
|fuelType  |  What type of fuel the car uses.|
|brand  |  The brand of the car.|
|notRepairedDamage  |  If the car has a damage which is not yet repaired.|
dateCreated  |  The date on which the eBay listing was created.|
nrOfPictures  |  The number of pictures in the ad.|
postalCode  |  The postal code for the location of the vehicle.|
lastSeenOnline  |  When the crawler saw this ad last online.|


---------------


### Cleaning Column Names

We notice that the columns uses a `camelcase` instead of the prefered Python's `snakecase`.
In this section, we will **`convert the column names from camelcase to snakecase`** and **`reword some of the column names`** based on the data dictionary to be more descriptive.

In [5]:
# display an array of the existing column names.
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [6]:
# modify the columns names
new_columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_of_pictures', 'postal_code',
       'last_seen']
autos.columns = new_columns

In [7]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We have succeeded to modify the name of some columns to the prefered Python's snakecase. These columns include but not limited to;


> * `yearOfRegistration` to `registration_year`
> * `monthOfRegistration` to `registration_month`
> * `notRepairedDamage` to `unrepaired_damage`
> * `dateCreated` to `ad_created` 

----------------

### Initial Exploration and Cleaning.

In this section, we will be doing some basic data exploration to determine what other cleaning tasks need to be done.
Initially we will look for:
* Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
* Examples of numeric data stored as text which can be cleaned and converted.

In [8]:
# get description of the dataset for both categorical and numeric columns
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


From the output above,
* We see that the `seller` and `offer_type` columns have mostly one value, making them candidates to be dropped
* Also, notice that the `price` and `odometer` columns contain numeric values stored as text. we will thus need to convert them.
* We will also rename the `odometer column to odometer_km` to make it more descriptive 

In [9]:
# Remove any non-numeric character and convert to float

autos["price"] = autos["price"].str.replace("$", "", regex=False)
autos["price"] = autos["price"].str.replace(",", "", regex=False).astype(float)

autos["odometer"] = autos["odometer"].str.replace("km", "", regex=False)
autos["odometer"] = autos["odometer"].str.replace(",", "", regex=False).astype(float)


In [10]:
# Rename the odometer column to odometer_km
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

# display dataframe info to be sure we made our changes
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date_crawled        50000 non-null  object 
 1   name                50000 non-null  object 
 2   seller              50000 non-null  object 
 3   offer_type          50000 non-null  object 
 4   price               50000 non-null  float64
 5   abtest              50000 non-null  object 
 6   vehicle_type        44905 non-null  object 
 7   registration_year   50000 non-null  int64  
 8   gearbox             47320 non-null  object 
 9   power_ps            50000 non-null  int64  
 10  model               47242 non-null  object 
 11  odometer_km         50000 non-null  float64
 12  registration_month  50000 non-null  int64  
 13  fuel_type           45518 non-null  object 
 14  brand               50000 non-null  object 
 15  unrepaired_damage   40171 non-null  object 
 16  ad_c

### Exploring the Odometer and Price Columns

 We continue exploring the data, specifically looking for data that doesn't look right.
Here's the steps we'll take:

Analyze the columns and look for any values that looks unrealistically high or low (outliers) that we might want to remove.

#### Exploring the Price column

In [11]:
# See how many unique values in the price column
print("No. of price unique values:", autos["price"].unique().shape)

# See how many unique values in the price column
print("\nNo. of odometer_km unique values:",autos["odometer_km"].unique().shape)

No. of price unique values: (2357,)

No. of odometer_km unique values: (13,)


In [12]:
# View min/max/median/mean values in the price column
print( autos["price"].describe())

print()
# View min/max/median/mean values in the odometer_km column
print(autos["odometer_km"].describe())

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


Looking at the output, we notice in the `price` column, that atleast 75% of the data are `less than 10000` however, we find the maximum value to be about `100 million`. This is a clear indication of an outlier. Let's further explore these columns to be sure of the existence of outlier

In [13]:
# get the highest unique price value with their counts
print(autos["price"].value_counts().sort_index(ascending=False).head(10))

print()
# get the highest unique odometer_km value with their counts
print(autos["odometer_km"].value_counts().sort_index(ascending=False).head(5))

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
Name: price, dtype: int64

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
Name: odometer_km, dtype: int64


In [14]:
# get the lowest unique price value with their counts
print(autos["price"].value_counts().sort_index(ascending=True).head())

print()
# get the lowest unique odometer_km value with their counts
print(autos["odometer_km"].value_counts().sort_index(ascending=True).head())

0.0    1421
1.0     156
2.0       3
3.0       1
5.0       2
Name: price, dtype: int64

5000.0     967
10000.0    264
20000.0    784
30000.0    789
40000.0    819
Name: odometer_km, dtype: int64


In [15]:
# We drop all rows with price of 1 million and above which we consider outliers
autos = autos[autos["price"] < 1000000]

In [16]:
# get number of rows left after dropping outliers in price column
autos.shape[0]

49989

In [17]:
# View min/max/median/mean values in the price column after dropping outliers
print( autos["price"].describe())

count     49989.000000
mean       5781.194763
std       11833.263624
min           0.000000
25%        1100.000000
50%        2950.000000
75%        7200.000000
max      999999.000000
Name: price, dtype: float64


From the result, we can clearly see that the outliers in the price column which constituted `less than 0.05%` greatly skewed the mean. Removing these values has moved out mean price from `9840` to `5781`.
This is a great improvement, given that we have lost just 11 rows.

### Exploring the date columns

In this section, we are moving on to the date columns to understand the date range the data covers. We saw that there are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

The `registration_month` and `registration_year` columns are represented as numeric values, while the other three columns are identified as string values.

In [18]:
autos[["date_crawled", "ad_created", "last_seen"]].tail()

,date_crawled,ad_created,last_seen
49995,2016-03-27 14:38:19,2016-03-27 00:00:00,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,2016-03-28 00:00:00,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,2016-04-02 00:00:00,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,2016-03-08 00:00:00,2016-04-05 16:45:07
49999,2016-03-14 00:42:12,2016-03-13 00:00:00,2016-04-06 21:18:48


Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively.

You'll notice that the first 10 characters represent the day (e.g. `2016-03-12`). We will extract just the date values.

In [19]:
# extract just the date values

date_crawled = autos["date_crawled"].str[:10]
last_seen = autos["last_seen"].str[:10]
ad_created = autos["ad_created"].str[:10]

In [20]:
# display first five rows of the extracted date values

print(date_crawled.head())

print()
print(last_seen.head())

print()
print(ad_created.head())

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: date_crawled, dtype: object

0    2016-04-06
1    2016-04-06
2    2016-04-06
3    2016-03-15
4    2016-04-01
Name: last_seen, dtype: object

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: ad_created, dtype: object


To understand the date range, we extracted just the date values from the `date_crawled`, `last_seen`, and `ad_created` columns.

Next, calculate the distribution of values in these columns (all string columns) as percentages.

In [21]:
# include missing values in the distribution and use percentages instead of counts
# rank by date in ascending order (earliest to latest)

date_crawled.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025386
2016-03-06    0.013943
2016-03-07    0.035968
2016-03-08    0.033267
2016-03-09    0.033207
2016-03-10    0.032127
2016-03-11    0.032487
2016-03-12    0.036768
2016-03-13    0.015563
2016-03-14    0.036628
2016-03-15    0.033987
2016-03-16    0.029506
2016-03-17    0.031527
2016-03-18    0.013063
2016-03-19    0.034908
2016-03-20    0.037828
2016-03-21    0.037508
2016-03-22    0.032907
2016-03-23    0.032387
2016-03-24    0.029106
2016-03-25    0.031747
2016-03-26    0.032487
2016-03-27    0.031047
2016-03-28    0.034848
2016-03-29    0.034168
2016-03-30    0.033627
2016-03-31    0.031907
2016-04-01    0.033807
2016-04-02    0.035408
2016-04-03    0.038689
2016-04-04    0.036488
2016-04-05    0.013103
2016-04-06    0.003181
2016-04-07    0.001420
Name: date_crawled, dtype: float64

Looking at the distribution of the `date_crawled` column, we notice the crawling started on `2016-03-05`(with a percentage distribution of `2.5386%` for this date) and ended on `2016-04-07`(with a percentage distribution of `0.142%`) which also happens to be the month with the lowest number of crawled. This distribution sees its maximum on `2016-04-03`(with a percentage distribution of `3.8689%` for this date)

---------------

In [22]:
ad_created.value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000020
2015-08-10    0.000020
2015-09-09    0.000020
2015-11-10    0.000020
2015-12-05    0.000020
                ...   
2016-04-03    0.038929
2016-04-04    0.036848
2016-04-05    0.011843
2016-04-06    0.003261
2016-04-07    0.001280
Name: ad_created, Length: 76, dtype: float64

In [23]:
# Sort the ad_created columns by percentage distrinution
ad_created.value_counts(normalize=True, dropna=False).sort_values(ascending=False)

2016-04-03    0.038929
2016-03-20    0.037868
2016-03-21    0.037708
2016-04-04    0.036848
2016-03-12    0.036608
                ...   
2015-12-30    0.000020
2016-01-16    0.000020
2016-02-07    0.000020
2016-01-13    0.000020
2016-02-16    0.000020
Name: ad_created, Length: 76, dtype: float64

Looking at the distribution of the `ad_created` column, we notice the creation of ads on the website started on `2015-06-11`(with a percentage distribution of `0.002%` for this date) and ended on `2016-04-07`(with a percentage distribution of `0.128%`). This distribution saw its maximum on `2016-04-03`(with a percentage distribution of `3.8929%` for this date).

-------------

In [24]:
last_seen.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001080
2016-03-06    0.004421
2016-03-07    0.005361
2016-03-08    0.007582
2016-03-09    0.009842
2016-03-10    0.010762
2016-03-11    0.012523
2016-03-12    0.023805
2016-03-13    0.008982
2016-03-14    0.012803
2016-03-15    0.015883
2016-03-16    0.016444
2016-03-17    0.027926
2016-03-18    0.007422
2016-03-19    0.015743
2016-03-20    0.020705
2016-03-21    0.020725
2016-03-22    0.021585
2016-03-23    0.018584
2016-03-24    0.019564
2016-03-25    0.019204
2016-03-26    0.016964
2016-03-27    0.016024
2016-03-28    0.020865
2016-03-29    0.022345
2016-03-30    0.024845
2016-03-31    0.023825
2016-04-01    0.023105
2016-04-02    0.024885
2016-04-03    0.025366
2016-04-04    0.024625
2016-04-05    0.124267
2016-04-06    0.220989
2016-04-07    0.130949
Name: last_seen, dtype: float64

From the distribution of the `last_seen` column, we notice the earliest last seen date on the website was on `2016-03-05`(with a percentage distribution of `0.1080%` which is the minimum distribution) and the latest last seen date is on `2016-04-07`(with a percentage distribution of `13.0949%`). This distribution saw its maximum on `2016-04-06`(with a percentage distribution of `22.0989%` for this date).

--------

In [25]:
# get the distribution of registration_year
autos["registration_year"].describe()

count    49989.000000
mean      2005.074916
std        105.724083
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The result the distribution of the year of registration shows a lot of inconsistences in the data. The minimum(`1000`) as well as the maximum(`9999`) year of registration are all inaccurate values. Some of these values don't make sense to us.
This therefore mean we will hve to further explore this column and possibly clean these values.

--------

### Dealing with Incorrect Registration Year Data

One thing that stands out from the exploration we did above is that the registration_year column contains some odd values:

> * The minimum value is 1000, before cars were invented.
> * The maximum value is 9999, many years into the future.

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. 

Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s. Therefore we will start by counting the number of listings with cars that fall outside the `1900 - 2016` interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [26]:
# get registration year outside 1900 - 2016
inaccurate_reg_year = autos[(autos["registration_year"] < 1900) | (autos["registration_year"] > 2016)]

inaccurate_reg_year["registration_year"].value_counts()

2017    1452
2018     491
9999       4
5000       4
2019       3
9000       2
1800       2
9996       1
6200       1
4100       1
4500       1
1000       1
1111       1
8888       1
2800       1
5911       1
1500       1
4800       1
1001       1
Name: registration_year, dtype: int64

Looking at the values counts, there are only `5` registration year listing earlier than 1900. Also as explained above, a car can't be first registered after the listing was seen, which mean any vehicle with a registration year above 2016 is definitely inaccurate. 

We will therefore retain only those rows with registration year between `1900 and 2016`

In [27]:
# Retain only data with registration year between 1900 and 2016

autos_clean = autos[(autos["registration_year"]>=1900) & (autos["registration_year"]<=2016)]

In [28]:
autos_clean.shape

(48019, 20)

In [29]:
# calculate the distribution of the registration year between 1900 - 2016

autos_clean["registration_year"].value_counts(normalize=True).sort_index()

1910    0.000187
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000042
          ...   
2012    0.027552
2013    0.016785
2014    0.013849
2015    0.008309
2016    0.027406
Name: registration_year, Length: 78, dtype: float64

Now we have removed all the values outside 1900 - 2016 and looking at our distribution now, it make sense given that there registration year now is in the range 1910 to 2016. Also, looking at the percentage distribution, we see that the top registered cars were between 1999 and 2005. The very low distribution values in the early 1900's also makes sense as fewer cars were produced then.

--------

### Exploring Price by Brand

In this section, we will explore the prices across different car brands. However, we will only select top `20 brands` to explore. 

In [30]:
# identify the unique brands in our dataset and their counts

autos_clean["brand"].value_counts().sort_values(ascending=False)

volkswagen        10187
bmw                5283
opel               5194
mercedes_benz      4579
audi               4149
ford               3351
renault            2274
peugeot            1418
fiat               1242
seat                873
skoda               770
mazda               727
nissan              725
smart               668
citroen             668
toyota              599
sonstige_autos      523
hyundai             473
volvo               444
mini                415
mitsubishi          391
honda               377
kia                 341
alfa_romeo          318
porsche             293
suzuki              284
chevrolet           274
chrysler            176
daihatsu            123
dacia               123
jeep                108
subaru              105
land_rover           98
saab                 77
jaguar               76
trabant              75
daewoo               72
rover                65
lancia               52
lada                 29
Name: brand, dtype: int64

In [31]:
print(len(autos_clean["brand"].value_counts()))

40


In [32]:
# select top 20 brands
top_20_brands = autos_clean["brand"].value_counts().sort_values(ascending=False).index[:20]

print(top_20_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'mazda', 'nissan', 'smart',
       'citroen', 'toyota', 'sonstige_autos', 'hyundai', 'volvo', 'mini'],
      dtype='object')


There are `40` car brands in our dataset. From the counts, we notice that the top brand is `volkswagen(10187)`, followed by `bmw(5283)` and `opel(5194)`, while the least common brands are `rover(65)`, `lancia(52)` and `lada(29)`. We have selected the `top 20` brands in the dataset which we will use to perform our aggregation.

In [33]:
# create empty dictionary to hold your aggregate data
brand_mean_price = {}

# Loop over selected brands, and assign the mean price to the dictionary
for b in top_20_brands:
    selected_rows = autos_clean[autos_clean["brand"] == b]
    mean_price = selected_rows["price"].mean()
    brand_mean_price[b] = mean_price

In [34]:
for key, value in brand_mean_price.items():
    print("{} : ${:,.2f}".format(key, value))

volkswagen : $5,426.38
bmw : $8,102.54
opel : $2,876.72
mercedes_benz : $8,485.24
audi : $9,093.65
ford : $3,949.42
renault : $2,395.42
peugeot : $3,039.47
fiat : $2,711.80
seat : $4,296.49
skoda : $6,334.92
mazda : $4,010.77
nissan : $4,664.89
smart : $3,542.71
citroen : $3,699.94
toyota : $5,115.33
sonstige_autos : $10,805.08
hyundai : $5,308.54
volvo : $4,757.11
mini : $10,460.01


From the result above, we notice that the brand with the highest price is `audi($9,093.65)`, followed by `mercedes_benz($8,485.24)` and `bmw($8,102.54)`.
These brands constitute the top 3 most expensive brands, while `opel($2,876.72)` and `ford($3,949.42)` are the least expensive

----------------

### Storing Aggregate Data in a DataFrame

For the top 6 brands, we will use aggregation to understand the average mileage for those cars and if there's any visible link with mean price.
While our natural instinct may be to display both aggregated series objects and visually compare them, this has a few limitations:

* it's difficult to compare more than two aggregate series objects if we want to extend to more columns
* we can't compare more than a few rows from each series object
* we can only sort by the index (brand name) of both series objects so we can easily make visual comparisons

Instead, we can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly.

In [35]:
# create empty dictionary to hold your aggregate data
brand_mean_mileage = {}

# Loop over selected brands, and assign the mean mileage to the dictionary
for b in top_20_brands:
    selected_rows = autos_clean[autos_clean["brand"] == b]
    mean_mileage = selected_rows["odometer_km"].mean()
    brand_mean_mileage[b] = mean_mileage

In [36]:
# Convert both brand_mean_price and brand_mean_mileage dictionaries to series objects

bmp_series = pd.Series(brand_mean_price)
bmm_series = pd.Series(brand_mean_mileage)

In [37]:
# Create a dataframe from the bmp_series
brand_df = pd.DataFrame(bmp_series, columns=["mean_price"])

# Assign bmm_series as a new column in this dataframe
brand_df["mean_mileage"] = bmm_series

In [38]:
brand_df.sort_values("mean_price", ascending=False)

,mean_price,mean_mileage
sonstige_autos,10805.078394,87466.539197
mini,10460.012048,88602.409639
audi,9093.650036,129287.780188
mercedes_benz,8485.239572,130856.082114
bmw,8102.536248,132431.383684
skoda,6334.919481,110954.545455
volkswagen,5426.382546,128728.281143
hyundai,5308.539112,106511.627907
toyota,5115.333890,115709.515860
volvo,4757.108108,138355.855856


### Conclusion

Looking at the dataframe we just created above, we can observe that for the top 6 brands, we note that there is an increase in the mileage as the price drops down. This is a clear indication that `price is directly related to the mileage`